## Imports

In [1]:
from pymol.Qt import QtCore
import attrs
import os
import pathlib
from typing import Iterable
import functools
import enum

PYQT_SIGNAL = QtCore.pyqtSignal
PYQT_SLOT = QtCore.pyqtSlot
PYQT_OBJECT = QtCore.QObject

## Dynamically generated signals

### Introspection

In [ ]:
# from https://stackoverflow.com/a/57295098
# apparently out of date

def get_signals(source):
    cls = source if isinstance(source, type) else type(source)
    signal = type(QtCore.pyqtSignal())
    for subcls in cls.mro():
        clsname = f'{subcls.__module__}.{subcls.__name__}'
        for key, value in sorted(vars(subcls).items()):
            if isinstance(value, signal):
                print(f'{key} [{clsname}]')

def list_all_signals(obj):
    attr_names = dir(obj)
    attributes = (getattr(obj, attr_name) for attr_name in attr_names)
    connectable = filter(lambda l: hasattr(l, "connect"), attributes)
    return connectable

class SignalListener(QtCore.QObject):
    @QtCore.pyqtSlot()
    def universal_slot(self, *args, **kwargs):
        print("Signal caught" + 30 * "-")
        print("sender:", self.sender())
        meta_method = (
            self.sender().metaObject().method(self.senderSignalIndex())
        )
        print("signal:", meta_method.name())
        print("signal signature:", meta_method.methodSignature())

SIGNAL_LISTENER = SignalListener()

def spy_on_all_signals(obj, listener = SIGNAL_LISTENER):
    for signal in list_all_signals(obj):
        signal.connect(SIGNAL_LISTENER.universal_slot)

### Simplest case - no dynamic stuff

In [ ]:
class TestModel0(QtCore.QObject):
    _y_changed = QtCore.pyqtSignal(str, name="_y_changed")
    def __init__(self, y):
        super(TestModel0, self).__init__()
        self._y = y
        
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, value):
        self._y = value
        print("set y")
        self._y_changed.emit(value)
        print('done')

class TestControl0(QtCore.QObject):
    def __init__(self, model):
        super(TestControl0, self).__init__()
        self.model = model
        self.model._y_changed.connect(self.on_y_changed)
        
    @QtCore.pyqtSlot(str)
    def on_y_changed(self, val):
        print("caught y changed", val)
        
m0 = TestModel0("baz")
t0 = TestControl0(m0)

In [ ]:
m0.y = 'newstr'

In [ ]:
class TestModel01(TestModel0):
    pass

hasattr(TestModel01, "_y_changed")

In [ ]:
m01 = TestModel01("baz")
t01 = TestControl0(m01)

In [ ]:
m01.y = 'newstr'

In [ ]:
m01.dumpObjectInfo()

### initial implementation of dynamic Signals - BROKEN

In [ ]:
PYQT_SIGNAL = QtCore.pyqtSignal
PYQT_SLOT = QtCore.pyqtSlot
PYQT_OBJECT = QtCore.QObject

class SignalWrapper():
    """Descriptor to automatically emit a pyqtSignal (assumed predefined)
    on change of a model attribute.
    """
    def __init__(self, name, signal_type=None):
        self.__set_name__(None, name)
        self.signal_type = signal_type

    @staticmethod
    def _private_from_public_name(name):
        return '_' + name

    @staticmethod
    def _public_from_private_name(name):
        assert name.startswith('_')
        return name[1:]

    @staticmethod
    def _signal_from_public_name(name):
        return '_' + name + '_changed'

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = self._private_from_public_name(name)
        self.signal_name = self._signal_from_public_name(name)

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        """Emit `signal_name` when value is changed to a new value (only.)
        """
        old_value = getattr(obj, self.private_name)
        try:
            setattr(obj, self.private_name, value)
            if old_value != value:
                self._emit(obj, value)
        except ValueError:
            # if attrs validation fails, don't emit signal
            raise

    def _emit(self, obj, value=None):
        """Emit associated signal with current attribute value.
        """
        try:
            if value is None:
                value = getattr(obj, self.private_name)
            getattr(obj, self.signal_name).emit(self.signal_type(value))
        except ValueError:
            # if attrs validation fails, don't emit signal
            raise

In [ ]:
def _attr_field_transformer(cls, fields):
    """Modify field definition process on attrs/dataclass in order to set up
    signal and signal descriptor. The original field is mapped to `private_name`,
    while the descriptor is assigned to the original `public_name`.

    See https://www.attrs.org/en/stable/extending.html#automatic-field-transformation-and-modification.
    """
    new_fields = []
    for f in fields:
        # really don't want to put all signal type-casting logic here
        if attrs.has(f.type):
            # don't create a descriptor/signal for attributes that are other
            # Models (ie building up Model object through composition.)
            continue

        if isinstance(f.type, pathlib.Path):
            signal_type = str
        elif isinstance(f.type, enum.Enum):
            signal_type = int
        else:
            signal_type = f.type

        desc = SignalWrapper(f.name, signal_type)
        renamed_f = f.evolve(name=desc.private_name)
        new_fields.append(renamed_f)
        setattr(cls, desc.public_name, desc)
        setattr(cls, desc.signal_name, PYQT_SIGNAL(signal_type, name=desc.signal_name))
    return new_fields

def attrs_define_w_signals(cls=None, **deco_kwargs):
    """Wrap the attrs.define() class decorator to automatically invoke
    `_attr_field_transformer`, to automatically define and emit signals when the
    values of the fields of `cls` are changed.
    """
    deco_kwargs.update({"slots":False, "field_transformer":_attr_field_transformer})
    if cls is None:
        # decorator called without arguments
        return functools.partial(attrs_define_w_signals, **deco_kwargs)

    # check that the class we're decorating is capable of emitting Signals
    # assert any(issubclass(cls_, PYQT_OBJECT) for cls_ in cls.__mro__)

    # attrs auto-generates an __init__ method; need to manually ensure that
    # super() is called.
    # https://www.attrs.org/en/stable/init.html#hooking-yourself-into-initialization
    def _pre_init(self):
        super(cls, self).__init__()
    setattr(cls, "__attrs_pre_init__", _pre_init)

    # hacky but necessary way to sync up associated Views with the Model. Model
    # needs to be instantiated before it's connect()ed to views, but this means
    # views don't know about inital values of model fields. To fix this, provide
    # a method to manually fire all _*_changed signals for all model fields.
    def _on_connect(self):
        cls_ = type(self)
        for f in attrs.fields(cls_):
            private_name = f.name # _attr_field_transformer remapped names
            public_name = SignalWrapper._public_from_private_name(private_name)
            signal_name = SignalWrapper._signal_from_public_name(public_name)
            if hasattr(cls_, signal_name):
                # descriptors are class attributes; call method on the class
                getattr(cls_, public_name)._emit(self)
    setattr(cls, "on_connect", _on_connect)

    # apply the attrs dataclass decorator, with descriptors and signals assigned
    # according to _attr_field_transformer()
    return attrs.define(cls, **deco_kwargs)

In [ ]:
@attrs_define_w_signals
class TestModel1(PYQT_OBJECT):
    y:str
    
m1 = TestModel1(y="foo")
[print(s) for s in list_all_signals(m1)]

In [ ]:
spy_on_all_signals(m1)

In [ ]:
class TestControl1(PYQT_OBJECT):
    def __init__(self, model):
        super(TestControl1, self).__init__()
        self.model = model
        self.model._y_changed.connect(self.on_y_changed)
        
    @PYQT_SLOT(str)
    def on_y_changed(self, val):
        print("caught y changed", val)

t1 = TestControl1(m1)

In [ ]:
m1.y = "blerg"

In [ ]:
m1.dumpObjectInfo()

In [ ]:

class TestModel11(TestModel1):
    pass
print(hasattr(TestModel1, "_y_changed"), hasattr(TestModel11, "_y_changed"))

In [ ]:
m11 = TestModel11(y="foo")

In [ ]:
m11.dumpObjectInfo()

In [ ]:
type(TestModel11)

In [ ]:
@attrs_define_w_signals
class TestModel12():
    y:str
    
class TestModel13(QtCore.QObject, TestModel12):
    pass

In [ ]:
print(hasattr(TestModel13, "_y_changed"))
m13 = TestModel13(y="foo")
m13.dumpObjectInfo()

### Second try: metaclass implementation

Based on https://stackoverflow.com/a/66266877

In [ ]:
class TestMetaclass(type(object)):
    def __new__(cls, name, bases, attrs_):
        print(attrs_)
        return super().__new__(cls, name, bases, attrs_)
    
@attrs.define
class TestAttrsClass(metaclass=TestMetaclass):
    x: int
    y: str

So custom metaclass *does* get called after decorator, apparently only if no decorator kwargs.

In [ ]:
attrs.fields(TestAttrsClass)

In [ ]:
class PropertyWrapper(QtCore.pyqtProperty):
    """Property implementation: gets, sets, and notifies of change."""
    def __init__(self, type_, name, notify):
        super().__init__(type_, self.getter, self.setter, notify=notify)
        self.name = name
        self.signal_type = type_
        print(f'\tProp {name} signal type {type_} {type(type_)}')
        
    @staticmethod
    def _private_from_public_name(name):
        return '_' + name.lstrip('_')

    @staticmethod
    def _public_from_private_name(name):
        assert name.startswith('_')
        return name.lstrip('_')

    @staticmethod
    def _signal_from_public_name(name):
        return '_' + name.lstrip('_') + '_changed'
    
    @staticmethod
    def _signal_from_private_name(name):
        return '_' + name.lstrip('_') + '_changed'
    
    @property
    def private_name(self):
        return self._private_from_public_name(self.name)
    
    @property
    def signal_name(self):
        return self._signal_from_public_name(self.name)

    def getter(self, instance):
        return getattr(instance, self.private_name)

    def setter(self, instance, value):
        signal = getattr(instance, self.signal_name)
        if type(value) in (list, dict):
            value = _MAKE_NOTIFIED(value, signal)
            signal.emit(value)
        else:
            # coerce from field value
            foo = getattr(instance, self.private_name)
            print(f'\tProp {self.name} value {foo} {type(foo)}')
            print(f'\tProp {self.name} signal type {type_} {type(type_)}')
            old_value = self.signal_type(getattr(instance, self.private_name))
            if old_value != value:
                signal.emit(self.signal_type(value)) # may be redundant
        setattr(instance, self.private_name, value)

def _attr_field_transformer(cls, fields):
    """Modify field definition process on attrs/dataclass in order to set up
    signal and signal descriptor. The original field is mapped to `private_name`,
    while the descriptor is assigned to the original `public_name`.

    See https://www.attrs.org/en/stable/extending.html#automatic-field-transformation-and-modification.
    """
    new_fields = []
    for f in fields:
        assert not attrs.has(f.type)
            # don't create a descriptor/signal for attributes that are other
            # Models (ie building up Model object through composition.)
            # currently can't handle this case.
        private_name = PropertyWrapper._private_from_public_name(f.name)
        new_fields.append(f.evolve(name=private_name))
    return new_fields

def attrs_define(cls=None, **deco_kwargs):
    deco_kwargs.update({"slots":True, "field_transformer":_attr_field_transformer})
    if cls is None:
        # decorator called without arguments
        return functools.partial(attrs_define, **deco_kwargs)

    # attrs auto-generates an __init__ method; need to manually ensure that
    # super() is called.
    # https://www.attrs.org/en/stable/init.html#hooking-yourself-into-initialization
    def _pre_init(self):
        QtCore.QObject.__init__(self)
    setattr(cls, "__attrs_pre_init__", _pre_init)

    # apply the attrs dataclass decorator, with descriptors and signals assigned
    # according to _attr_field_transformer()
    return attrs.define(cls, **deco_kwargs)

_AUTOSIGNAL_TYPE_COERCE = {
    list: 'QVariantList', dict: 'QVariantMap',
    pathlib.Path: str,
    enum.Enum: int
}

class AutoSignalMetaclass(type(QtCore.QObject)):
    """Lets a class succinctly define Qt properties."""
    def __new__(cls, name, bases, attrs_):
        if '__attrs_attrs__' not in attrs_:
            print("test2")
            # first call, before attrs decorator; ordinary Object behavior
            return type.__new__(cls, name, bases, attrs_)
        
        print("test")
        # If we get here, attrs decorator has done its work
        for f in attrs_['__attrs_attrs__']:
            signal_type = _AUTOSIGNAL_TYPE_COERCE.get(f.type, f.type)
            
            private_name = f.name # _attr_field_transformer remapped names
            public_name = PropertyWrapper._public_from_private_name(private_name)
            signal_name = PropertyWrapper._signal_from_public_name(public_name)
            print(f'\tAdding signal {signal_name} {signal_type}')
            signal = QtCore.pyqtSignal(signal_type, name=signal_name)
            attrs_[signal_name] = signal
            attrs_[public_name] = PropertyWrapper(type_=signal_type, name=public_name, notify=signal)
    
        # hacky but necessary way to sync up associated Views with the Model. Model
        # needs to be instantiated before it's connect()ed to views, but this means
        # views don't know about inital values of model fields. To fix this, provide
        # a method to manually fire all _*_changed signals for all model fields.
        def _on_connect(self):
            cls_ = type(self)
            for f in attrs.fields(cls_):
                private_name = f.name # _attr_field_transformer remapped names
                public_name = PropertyWrapper._public_from_private_name(private_name)
                signal_name = PropertyWrapper._signal_from_public_name(public_name)
                if hasattr(cls_, signal_name):
                    # signals are class attributes BUT need to call emit() on the instance
                    value = getattr(self, public_name)
                    getattr(self, signal_name).emit(value)
        attrs_["on_connect"] = _on_connect
            
        return super().__new__(cls, name, bases, attrs_)

In [ ]:
@attrs_define
class TestModel2(QtCore.QObject, metaclass=AutoSignalMetaclass):
    x:int
    y:str
    
m2 = TestModel2(x=2, y="foo")
[print(s) for s in list_all_signals(m2)]

In [ ]:
m2.as_dict()

In [ ]:
attrs.asdict(m2)

In [ ]:
class TestControl2(QtCore.QObject):
    def __init__(self, model):
        super(TestControl2, self).__init__()
        self.model = model
        self.model._y_changed.connect(self.on_y_changed)
        
    @PYQT_SLOT(str)
    def on_y_changed(self, val):
        print("caught y changed", val)

t2 = TestControl2(m2)

In [ ]:
m2.y = "now test"

In [ ]:
m2.on_connect()

In [ ]:
m2._y_changed.emit("arf")

In [ ]:
# now add list/dict mutability signals from SO
class MakeNotified:
    """Adds notifying signals to lists and dictionaries.
    
    Creates the modified classes just once, on initialization.
    """
    change_methods = {
        list: ['__delitem__', '__iadd__', '__imul__', '__setitem__', 'append',
               'extend', 'insert', 'pop', 'remove', 'reverse', 'sort'],
        dict: ['__delitem__', '__ior__', '__setitem__', 'clear', 'pop',
               'popitem', 'setdefault', 'update']
    }
    
    def __init__(self):
        if not hasattr(dict, '__ior__'):
            # Dictionaries don't have | operator in Python < 3.9.
            self.change_methods[dict].remove('__ior__')
        self.notified_class = {type_: self.make_notified_class(type_)
                               for type_ in [list, dict]}
    
    def __call__(self, seq, signal):
        """Returns a notifying version of the supplied list or dict."""
        notified_class = self.notified_class[type(seq)]
        notified_seq = notified_class(seq)
        notified_seq.signal = signal
        return notified_seq
    
    @classmethod
    def make_notified_class(cls, parent):
        notified_class = type(f'notified_{parent.__name__}', (parent,), {})
        for method_name in cls.change_methods[parent]:
            original = getattr(notified_class, method_name)
            notified_method = cls.make_notified_method(original, parent)
            setattr(notified_class, method_name, notified_method)
        return notified_class
    
    @staticmethod
    def make_notified_method(method, parent):
        @functools.wraps(method)
        def notified_method(self, *args, **kwargs):
            result = getattr(parent, method.__name__)(self, *args, **kwargs)
            self.signal.emit(self)
            return result
        return notified_method

_MAKE_NOTIFIED = MakeNotified()

In [ ]:
@attrs_define
class TestModel3(QtCore.QObject, metaclass=AutoSignalMetaclass):
    y:str
    x:list = attrs.Factory(list) # list[int] not handled correctly
    
m3 = TestModel3(y="foo", x=[1,2])
[print(s) for s in list_all_signals(m3)]

In [ ]:
class TestControl3(QtCore.QObject):
    def __init__(self, model):
        super(TestControl3, self).__init__()
        self.model = model
        self.model._y_changed.connect(self.on_y_changed)
        self.model._x_changed.connect(self.on_x_changed)
        
    @PYQT_SLOT(str)
    def on_y_changed(self, val):
        print("caught y changed", val)
        
    @PYQT_SLOT(list)
    def on_x_changed(self, val):
        print("caught x changed", val)

t3 = TestControl3(m3)

In [ ]:
m3.y = "test 1"

In [ ]:
m3.x = [4,5]

In [ ]:
m3.x.append(6)

In [ ]:
m3.x[0] = 420

In [ ]:
m3.on_connect()

Now test inheritance (no multiple inheritance)

In [ ]:
@attrs_define
class TestModel4(TestModel3):
    z:int = 0
    
m4 = TestModel4(y="foo", x=[1,2], z=7)
[print(s) for s in list_all_signals(m4)]

t4 = TestControl3(m4)

In [ ]:
m4.y = "test inherit"

In [ ]:
m4.x = [4,5]

In [ ]:
(hasattr(m4, '_TestModel4__x'), '_TestModel4__x' in m4.__dict__)

In [ ]:
'_TestModel4__x' in dir(m4)

In [ ]:
TestModel4.__attrs_attrs__

In [ ]:
(type(TestModel4.x), type(TestModel4._x))

In [ ]:
(type(TestModel3.x), type(TestModel3._x))

In [ ]:
TestModel4.__dict__

### add on_* methods to Controller classes

In [ ]:
foo = PYQT_SIGNAL(int)
foo.signatures

In [ ]:
class AutoSlotMetaclass(type(PYQT_OBJECT)):
    """Metaclass for dynamically associating PyQt Slots based on an associated Model.
    """
    def __new__(cls, name, bases, attrs_):
        if '_model_class' not in attrs_:
            return super().__new__(cls, name, bases, attrs_)
        model_cls = attrs_['_model_class']
        if not hasattr(model_cls, '__attrs_attrs__'):
            return super().__new__(cls, name, bases, attrs_)
        
        for f in model_cls.__attrs_attrs__:
            private_name = f.name # _attr_field_transformer remapped names
            public_name = PropertyWrapper._public_from_private_name(private_name)
            signal_name = PropertyWrapper._signal_from_public_name(public_name)
            signal_type = getattr(model_cls, public_name).signal_type
            method_name = 'on'+signal_name
            
            @QtCore.pyqtSlot(signal_type)
            def _slot_with_dummy_name(self, value):
                setattr(self.model, public_name, value)
                
            attrs_[method_name] = _slot_with_dummy_name
        return super().__new__(cls, name, bases, attrs_)

class BaseController(PYQT_OBJECT, metaclass = AutoSlotMetaclass):
    """Base class for our Controller classes.
    """
    def __init__(self, model, view):
        super(BaseController, self).__init__()
        self.model = model
        self.view = view

In [ ]:
class TestControl5(BaseController):
    _model_class = TestModel3

In [ ]:
'on_x_changed' in dir(TestControl5)

In [ ]:
m5 = TestModel3(y="foo", x=[1,2])
t5 = TestControl5(model=m5, view=None)
m5._x_changed.connect(t5.on_x_changed)
m5._y_changed.connect(t5.on_y_changed)

In [ ]:
m5.y = "test again"

In [ ]:

t4 = TestControl3(m4)
m4.on_connect()

## New Controller idea

In [ ]:
class TestControl1a(QtCore.QObject):
    x_valueChanged = PYQT_SIGNAL(str)

class TestControl1b(TestControl1a, QtCore.QObject):
    y_valueChanged = PYQT_SIGNAL(int)
    
    # only allowed to emit signals
    def xy_update(self, z):
        # uh oh
        # self.x_valueChanged.emit()
        


In [5]:
import typing
(isinstance(typing.Dict, typing.Mapping), issubclass(typing.Dict, typing.Mapping))

(False, True)

In [7]:

(isinstance(dict, typing.Mapping), issubclass(dict, typing.Mapping))

(False, True)

In [9]:
os.path.dirname("../foo/bar/baz.txt")

'../foo/bar'

In [10]:
os.path.expanduser("~")

'/Users/tsj'

In [14]:
foo = FooAttrs(x=13, bar = "bar")

TypeError: __init__() got an unexpected keyword argument 'bar'

In [13]:
@attrs.define(slots=True, kw_only=True)
class FooAttrs():
    x: int = 0
    
    def __attrs_post_init__(self, bar):
        self.bar = bar

### Property stuff

In [36]:
class PropertyWrapper(QtCore.pyqtProperty):
    """Wrapper for pyqtProperty which automatically emits signal on field value
    change.
    """
    def __init__(self, type_, name):
        # NB: pyqtProperty doesn't do anything with .notify; still need to 
        # .emit() the signal manually
        super().__init__(type_, self.getter, self.setter)
        self.name = name
        self.signal_type = type_

    @property
    def private_name(self):
        return '_'+self.name

    def getter(self, instance):
        return getattr(instance, self.private_name)

    def setter(self, instance, value):
        print(f"Setter called: {instance} {self.name}:{value}")
        setattr(instance, self.private_name, value)

def dummy_slot_deco(f):
    def _ff(*args):
        print(f"Decorator called: {args}")
        return f(args)
    return _ff
        
class Blerg(QtCore.QObject):
    x = PropertyWrapper(int, "x")
    
    def __init__(self, x):
        super().__init__()
        self._x = x
        self.x.setter = dummy_slot_deco(self.x.setter)

In [37]:
blerg = Blerg(3)

AttributeError: 'int' object has no attribute 'setter'

In [34]:
blerg.x = 6

In [35]:
print(blerg.x, blerg._x)

6 3


## Enum stuff

In [ ]:
class DummySignal():
    def __init__(self, type_):
        self._type = type_
        
    def emit(self, val):
        print("Emitted ", val)
        
class SignalWrapper():
    """Descriptor to automatically emit a pyqtSignal (assumed predefined)
    on change of a model attribute.
    """
    def __init__(self, name, signal_type=None):
        self.__set_name__(None, name)
        self.signal_type = signal_type

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name
        self.signal_name = '_' + name + '_changed'

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        """Emit `signal_name` when value is changed to a new value (only.)
        """
        old_value = getattr(obj, self.private_name)
        try:
            setattr(obj, self.private_name, value)
            if old_value != value:
                getattr(obj, self.signal_name).emit(self.signal_type(value))
        except ValueError:
            # if attrs validation fails, don't emit signal
            raise
            
def combo_enum_factory(enum_name, enum_vals_and_labels):
    """
    """
    _enum_cls = enum.Enum(enum_name, tuple(enum_vals_and_labels.keys()), start=0)
    desc = SignalWrapper(enum_name, signal_type=int)
    class cls_():
        def __init__(self, val):
            super().__init__()
            if isinstance(int, val):
                self._value = _enum_cls(val)
            else:
                self._value = _enum_cls[val]
    
    setattr(cls_, '_model', tuple(enum_vals_and_labels.values()))
    setattr(cls_, desc.signal_name, DummySignal(int))
    setattr(cls_, 'value', desc)
    return cls_

In [ ]:
Blerg = combo_enum_factory("Blerg", {"A": "A text", "B": "B text"})

In [ ]:
bar = Blerg.A

In [ ]:
str(fooE(1))

In [15]:
Foo = enum.Enum("Foo", "A B C")

In [16]:
type(Foo)

enum.EnumMeta

In [17]:
foo = Foo.A

In [18]:
dir(foo)

['__class__', '__doc__', '__module__', 'name', 'value']

In [19]:
foo.name

'A'

In [20]:
foo.value

1